In [6]:
!pip install langchain

  Using cached aiohttp-3.11.11-cp312-cp312-win_amd64.whl.metadata (8.0 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tenacity-9.0.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached aiohappyeyeballs-2.4.4-py3-none-any.whl.metadata (6.1 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached frozenlist-1.5.0-cp312-cp312-win_amd64.whl.metadata (14 kB)
  Using cached multidict-6.1.0-cp312-cp312-win_amd64.whl.metadata (5.1 kB)
  Using cached propcache-0.2.1-cp312-cp312-win_amd64.whl.metadata (9.5 kB)
  Using cached yarl-1.18.3-cp312-cp312-win_amd64.whl.metadata (71 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached zstandard-0.23.0-cp312-cp312-win_amd64.whl.metadata (3.0 

In [8]:
!pip install -U langchain-community

  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached pydantic_settings-2.7.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached mypy_extensions-1.0.0-py3-none-any.whl.metadata (1.1 kB)
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
   --------------------------------- ------ 2.1/2.5 MB 7.8 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 6.9 MB/s eta 0:00:00
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
Using cached httpx_sse-0.4.0-py3-none-any.whl (7.8 kB)
Using cached pydantic_settings-2.7.1-py3-none-any.whl (29 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
Using cached typing_inspect-0.9.0-py3-none-any.whl

In [1]:
import json
import boto3

In [2]:
# Load the JSON file
with open("aws_credentials.json", "r", encoding="utf-8-sig") as file:
    data = file.read()  # Read the file content
    credentials = json.loads(data) 

# Extract credentials
aws_access_key_id = credentials["aws_access_key_id"]
aws_secret_access_key = credentials["aws_secret_access_key"]

In [3]:
SERVICE_NAME = 'bedrock-runtime'
REGION_NAME = 'ap-south-1'

import boto3
bedrock = boto3.client(
 service_name=SERVICE_NAME,
 region_name=REGION_NAME,
 aws_access_key_id=aws_access_key_id,
 aws_secret_access_key=aws_secret_access_key,
 endpoint_url=f'https://{SERVICE_NAME}.{REGION_NAME}.amazonaws.com'
)

In [19]:
import os
from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain

In [24]:
from dotenv import load_dotenv

In [26]:
load_dotenv()

True

In [27]:
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
os.environ["NEO4J_URI"]=NEO4J_URI
os.environ["NEO4J_USERNAME"]=NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"]=NEO4J_PASSWORD


In [29]:
!pip install neo4j

  Using cached neo4j-5.27.0-py3-none-any.whl.metadata (5.9 kB)
Using cached neo4j-5.27.0-py3-none-any.whl (301 kB)


In [30]:
graph=Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,)

In [47]:
def extract_prompt(question):
    """Generate and return a tailored prompt based on the user's question."""
    prompt = f"""
    You are an intelligent assistant interacting with a Neo4j graph database. The user's question is:
    '{question}'
    Your goal is to:
    1. Understand the question's intent.
    2. Generate a Cypher query to retrieve relevant information just show output not more information.
    3. Respond in natural language with the results or appropriate feedback.
    4. If there are any questions are related to product then in product clause use 'contains'
    5. If there are any two questions use 'and' eg: MATCH (c:Company)-[:LOCATED_IN]->(location:Location),
      (c)-[:COMPANY_NAME]->(p:Product) WHERE location.country = 'country_name' AND p.name CONTAINS 'product_name' RETURN c.name
    """
    return prompt

In [49]:
from langchain.llms import Bedrock
from langchain.chains.graph_qa.cypher import GraphCypherQAChain

# Initialize LLM (not embeddings)
llm = Bedrock(model_id="mistral.mistral-7b-instruct-v0:2", client=bedrock)

# Initialize GraphCypherQAChain with LLM
chain = GraphCypherQAChain.from_llm(llm=llm, graph=graph, verbose=True,allow_dangerous_requests=True)

# Query
question = "How many companies located in India?"
prompt = extract_prompt(question)


response=chain.invoke({"query":prompt}) #y invoke.? or is there any other way to use invoke

print(response)




> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (c:Company)-[:LOCATED_IN]->(location:Location) WHERE location.country = 'India'
RETURN count(c) as num_companies

Full Context:
[{'num_companies': 9}]

> Finished chain.
{'query': "\n    You are an intelligent assistant interacting with a Neo4j graph database. The user's question is:\n    'How many companies located in India?'\n    Your goal is to:\n    1. Understand the question's intent.\n    2. Generate a Cypher query to retrieve relevant information just show output not more information.\n    3. Respond in natural language with the results or appropriate feedback.\n    4. If there are any questions are related to product then in product clause use 'contains'\n    5. If there are any two questions use 'and' eg: MATCH (c:Company)-[:LOCATED_IN]->(location:Location),\n      (c)-[:COMPANY_NAME]->(p:Product) WHERE location.country = 'country_name' AND p.name CONTAINS 'product_name' RETURN c.name\n    ", 'result':